# Recommendation System

Von  Jannis Breitenstein, Jonas Meyer, Verena Aschoff, Lucas Grewe, Thomas Wilmes <br>


Diese Notebook ist in folgende Abschnitte unterteilt:

+ [1. Import Bibliotheken](#1)<br>
+ [2. Laden und Aufbereiten der Filmdaten](#2)<br>
+ [2.1 Laden und Aufbereitung der Dten](#2.1)<br>
+ [2.2 Exportieren und Einlesen der DAten](#2.1)<br>
+ [3. Explorative Datenanalyse](#3)<br>
+ [3.1 Allgeime Daten über die Datensets](#3.1)<br>
+ [3.2 Veröffentlichung der Filme](#3.2)<br>
+ [3.3 Verteilung der Filmbewertungen](#3.3)<br>
+ [3.4 Wann sind die Filme bewertet worden](#3.4)<br>
+ [3.5 Verteilung von Filmbewertungen und Benutzern](#3.5)<br>
+ [4. Filtern der Daten](#4)<br>
+ [5. Anweden des KNN-Modells](#5)<br>
+ [5.1 Vorbereitung des KNN-Modells](#5.1)<br>
+ [5.2 Berechnung von allen Nachbarn und Export](#5.2)<br>
+ [5.3 Vorbereitung für Precision and Recall](#5.3)<br>
+ [5.4 Presicion and Recall](#5.4)<br>
+ [5.5 Empfehlung mit Hilfe des Fuzzy Algorithmus](#5.5)<br>
***

# <a id=1>1. Import Bibliotheken </a>


In [ ]:
# Benötigte Package
# Alternativ die Pakete installieren

import sys


## Es wird empfohlen, das Jupyter Notebook Paket zu installieren
!{sys.executable} -m pip install jupyter
###############################################

!{sys.executable} -m pip install pandas
!{sys.executable} -m pip install surprise
!{sys.executable} -m pip install numpy
!{sys.executable} -m pip install pytz
!{sys.executable} -m pip install keras
!{sys.executable} -m pip install scikit-surprise 

# Wird benötigt, wenn man dieses Notebook über Colabs benutzen möchte
#!{sys.executable} -m pip install google.colab
###############################################
!{sys.executable} -m pip install matplotlib
!{sys.executable} -m pip install plotly
!{sys.executable} -m pip install fastparquet
!{sys.executable} -m pip install seaborn
!{sys.executable} -m pip install pyarrow
!{sys.executable} -m pip install fuzzywuzzy
!{sys.executable} -m pip install python-Levenshtein
!{sys.executable} -m pip install scikit-learn
!{sys.executable} -m pip install scipy


In [ ]:
# Zum Speichern der Daten
import pandas as pd

# Wird zur Erstellung mehrdimensionalen Arrays benötigt
import numpy as np

# interaktive Diagramme erstellen
import matplotlib.pyplot as plt
import seaborn as sns

# interaktive Diagramme erstellen
from plotly.offline import init_notebook_mode, plot, iplot
import plotly.graph_objs as go
init_notebook_mode(connected=True)

# Zum Verschieben von Listen
from collections import deque

# Ähnlichkeiten zwischen Vektoren zu berechnen
from sklearn.neighbors import NearestNeighbors , BallTree, KDTree
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

# TO Fuzz
from fuzzywuzzy import fuzz

# Zum erstellen von Matrizen
from scipy.sparse import csr_matrix

import time
# Runtime variable
import json
from statistics import mean


***
# <a id=2>2. Laden und Aufbereiten der Filmdaten </a>

***

## 2.1 Datenaufbereitung </a>
***

In [ ]:
#Definieren der Dateipfade

# Enhält alle Filme mit Namen, Jahr und Film-ID.
# Ändern der Dateipfade für den entsprechenden Ordner.

combined_data_1 = './combined_data_1.txt'
combined_data_2 = './combined_data_2.txt'
combined_data_3 = './combined_data_3.txt'
combined_data_4 = './combined_data_4.txt'

#Auslesen der Daten aus den CSV-Dateien und mit neue Überschriften vergeben

combined_data_1_raw = pd.read_csv(combined_data_1, header=None, names=['Cust_Id', 'Rating', 'Date'], usecols=[0, 1, 2])
combined_data_2_raw = pd.read_csv(combined_data_2, header=None, names=['Cust_Id', 'Rating', 'Date'], usecols=[0, 1, 2])
combined_data_3_raw = pd.read_csv(combined_data_3, header=None, names=['Cust_Id', 'Rating', 'Date'], usecols=[0, 1, 2])
combined_data_4_raw = pd.read_csv(combined_data_4, header=None, names=['Cust_Id', 'Rating', 'Date'], usecols=[0, 1, 2])

## Aufgrunddessen, dass die Combined Data nicht nicht einfach verarbeitet können, müssen die Daten erst ausgelesen und als neue Datei gespeichert werden,
#  um in Zukunft einfacher auf die Daten zugreifen zu können.

# Suche nach leeren Zeilen, um das Dataframe für jeden Film aufzuschneiden
tmp_movies = combined_data_1_raw[combined_data_1_raw['Rating'].isna()]['Cust_Id'].reset_index()  
movie_indices = [[index, int(movie[:-1])] for index, movie in tmp_movies.values]

# Verschiebe die movie_indices um eins, um Start- und Endpunkte aller Filme zu erhalten
shifted_movie_indices = deque(movie_indices)
shifted_movie_indices.rotate(-1)


# Sammeln aller Dataframes
user_data = []

# Iterieren über alle Filme
for [df_id_1, movie_id], [df_id_2, next_movie_id] in zip(movie_indices, shifted_movie_indices):
    
    # Prüfen, ob es der letzte Film in der Datei ist
    if df_id_1<df_id_2:
        tmp_df = combined_data_1_raw.loc[df_id_1+1:df_id_2-1].copy()
    else:
        tmp_df = combined_data_1_raw.loc[df_id_1+1:].copy()
        
    # Spalte Movie_id erstellen
    tmp_df['Movie_Id'] = movie_id
    
    # Datenrahmen an die Liste anhängen
    user_data.append(tmp_df)

# Alle Dataframes zusammenfassen
df_1 = pd.concat(user_data)
del user_data, combined_data_1_raw, combined_data_1, tmp_movies, tmp_df, shifted_movie_indices, movie_indices, df_id_1, movie_id, df_id_2, next_movie_id

### Dieses Vorgehen wird für die anderen Dataframes wiederholt.

tmp_movies = combined_data_2_raw[combined_data_2_raw['Rating'].isna()]['Cust_Id'].reset_index() 
movie_indices = [[index, int(movie[:-1])] for index, movie in tmp_movies.values]

shifted_movie_indices = deque(movie_indices)
shifted_movie_indices.rotate(-1)

user_data = []


for [df_id_1, movie_id], [df_id_2, next_movie_id] in zip(movie_indices, shifted_movie_indices):
    
    if df_id_1<df_id_2:
        tmp_df = combined_data_2_raw.loc[df_id_1+1:df_id_2-1].copy()
    else:
        tmp_df = combined_data_2_raw.loc[df_id_1+1:].copy()
        
    tmp_df['Movie_Id'] = movie_id
    
    user_data.append(tmp_df)

df_2 = pd.concat(user_data)
del user_data, combined_data_2_raw, combined_data_2, tmp_movies, tmp_df, shifted_movie_indices, movie_indices, df_id_1, movie_id, df_id_2, next_movie_id


tmp_movies = combined_data_3_raw[combined_data_3_raw['Rating'].isna()]['Cust_Id'].reset_index() 
movie_indices = [[index, int(movie[:-1])] for index, movie in tmp_movies.values]

shifted_movie_indices = deque(movie_indices)
shifted_movie_indices.rotate(-1)

user_data = []

for [df_id_1, movie_id], [df_id_2, next_movie_id] in zip(movie_indices, shifted_movie_indices):
    
    if df_id_1<df_id_2:
        tmp_df = combined_data_3_raw.loc[df_id_1+1:df_id_2-1].copy()
    else:
        tmp_df = combined_data_3_raw.loc[df_id_1+1:].copy()
        
    tmp_df['Movie_Id'] = movie_id
    
    user_data.append(tmp_df)


df_3 = pd.concat(user_data)
del user_data, combined_data_3_raw, combined_data_3, tmp_movies, tmp_df, shifted_movie_indices, movie_indices, df_id_1, movie_id, df_id_2, next_movie_id

tmp_movies = combined_data_4_raw[combined_data_4_raw['Rating'].isna()]['Cust_Id'].reset_index()
movie_indices = [[index, int(movie[:-1])] for index, movie in tmp_movies.values]


shifted_movie_indices = deque(movie_indices)
shifted_movie_indices.rotate(-1)

user_data = []

for [df_id_1, movie_id], [df_id_2, next_movie_id] in zip(movie_indices, shifted_movie_indices):
    
    if df_id_1<df_id_2:
        tmp_df = combined_data_4_raw.loc[df_id_1+1:df_id_2-1].copy()
    else:
        tmp_df = combined_data_4_raw.loc[df_id_1+1:].copy()
        
    tmp_df['Movie_Id'] = movie_id
    
    user_data.append(tmp_df)

df_4 = pd.concat(user_data)
del user_data, combined_data_4_raw, combined_data_4, tmp_movies, tmp_df, shifted_movie_indices, movie_indices, df_id_1, movie_id, df_id_2, next_movie_id
print('Shape User-Ratings:\t{}'.format(df_4.shape))

#Zusammenfügen der aller Daten in einer Variable
data = [df_1, df_2,df_3,df_4]
df_rating = pd.concat(data)

# Löschen der Variablen
del df_1, df_2, df_3, df_4, data


In [ ]:
# Der nächste Schritt ist es, die Movie-Titel Datei zu importieren.
# Aufgrunddessen, dass Filme aus Seperatoren wie , oder ; enhalten, werden zunächst alle Daten in eine Spalte geladen und am Ende separiert. 
# Ändern der Dateipfade für den entsprechenden Ordner.

movie_tile_File = './movie_titles.csv'
########################################
df_movie_titles = pd.read_csv(movie_tile_File,
                           encoding = "ISO-8859-1",
                           delimiter= '\t',
                           header = None,
                           names = ['Ziel'])
                           # Speicher alle Daten erst in eine Spalte, danach trennt er diese
df_movie_titles[['Movie_Id', 'Year', 'Name']] = df_movie_titles['Ziel'].str.split(pat=",",n=2, expand=True)   

# Entfernen der Spalte Ziel
df_movie_titles= df_movie_titles.drop(['Ziel'], axis= 1)  

# Speichern des Dataframes in der entsprechenden Formatierung
Convert_dic= {'Movie_Id': 'int64','Year': 'object', 'Name': 'str'}
df_movie_titles = df_movie_titles.astype(Convert_dic)


del Convert_dic, movie_tile_File

***
## 2.2 Exportieren und Einlesen der Daten </a> 

***

In [ ]:
# Damit in Zukunft die Iteration nicht immer erneut Ausgeführt werden soll, werden die Daten in eine Parquet Datei geschrieben.
# Ändern der Dateipfade für den entsprechenden Ordner.
parquet_combined_data = './data_Comb.zip'
#########################################

df_rating.to_parquet(parquet_combined_data, index=False)
del parquet_combined_data

In [ ]:
#Einlesen der zuvor erstellen Parquet Datei  --> Rating-User-Movie_Id.
# Ändern der Dateipfade für den entsprechenden Ordner.
parquet_combined_data = './data_Comb.zip'
#############################################

df_rating = pd.read_parquet(parquet_combined_data)
# Formatieren des Dateiformats
Convert_dic_data= {'Cust_Id': 'int64', 'Rating': 'float32','Date': 'str', 'Movie_Id': 'Int64'}
df_rating= df_rating.astype(Convert_dic_data)
del parquet_combined_data, Convert_dic_data

In [ ]:
# Damit in Zukunft das Einlesen und das Trennen der Spalten nicht immer erneut ausgeführt werden soll, werden die Daten in eine Parquet Datei geschrieben.
# Ändern der Dateipfade für den entsprechenden Ordner.
parquet_movietitles_data = './movie_titles.zip'
##############################################
df_movie_titles.to_parquet(parquet_movietitles_data, index=False)
del parquet_movietitles_data

In [ ]:
#Einlesen der zuvor erstellen Parquet Datei.
# Ändern der Dateipfade für den entsprechenden Ordner.
parquet_movietitles_data = './movie_titles.zip'
#####################################################

df_movie_titles = pd.read_parquet(parquet_movietitles_data)
# Formatieren des Dateiformats
Convert_dic= {'Movie_Id': 'int64', 'Year': 'object', 'Name': 'str'}
df_movie_titles= df_movie_titles.astype(Convert_dic)
del parquet_movietitles_data, Convert_dic


***
# <a id=3>3. Explorative Datenanalyse </a>

***



## 3.1. Allgeime Daten über die Datensets </a>

In [ ]:
# Datenstruktur Filmbewertungen

display(df_rating)

#Anzahl der Benutzer
num_users = len(df_rating.Cust_Id.unique())

#Anzahl der eindeutigen Film-IDs
num_items = len(df_rating.Movie_Id.unique())
print('Es gibt {} eindeutige Users und {} eindeutige Filme im Dataframe'.format(num_users, num_items))

# Anzahl der Rating bei Gruppe
df_ratings_cnt_tmp = pd.DataFrame(df_rating.groupby('Rating').size(), columns=['count'])
df_ratings_cnt_tmp


# Anzahl der von jedem Nutzer abgegebenen Bewertung
df_users_cnt = pd.DataFrame(df_rating.groupby('Cust_Id').size(), columns=['count'])
df_users_cnt.head()


##  Ausgabe der Duplikate
dupli = df_rating.duplicated(keep=False).sum()
print('In dem Datensatz df_Rating liegen {} Duplikate vor'.format(dupli))

# Ausgabe der Datei Filmdaten

display(df_movie_titles)

del num_items, num_users, df_ratings_cnt_tmp, df_users_cnt, dupli

## 3.2. Veröffentlichung der Filme</a>
***

In [ ]:
# Daten einholen
data = df_movie_titles['Year'].value_counts().sort_index()

# Linie erstellen
trace = go.Scatter(x = data.index,
                   y = data.values,
                   marker = dict(color = '#db0000'))
# Layout erstellen
layout = dict(title = '{} Filme gruppiert nach Jahr der Veröffentlichung'.format(df_movie_titles.shape[0]),
              xaxis = dict(title = 'Erscheinungsjahr'),
              yaxis = dict(title = 'Filme'))

# Plot erstellen
fig = go.Figure(data=[trace], layout=layout)
iplot(fig)

del layout, trace, fig, data

***
## 3.3. Bewertung der Filme</a>
***

In [ ]:
# Daten einholen
data = df_rating['Rating'].value_counts().sort_index(ascending=False)

# Linie erstellen
trace = go.Bar(x = data.index,
               text = ['{:.1f} %'.format(val) for val in (data.values / df_rating.shape[0] * 100)],
               textposition = 'auto',
               textfont = dict(color = '#000000'),
               y = data.values,
               marker = dict(color = '#db0000'))
#  Layout erstellen
layout = dict(title = 'Verteilung {} User-Ratings'.format(df_rating.shape[0]),
              xaxis = dict(title = 'Rating'),
              yaxis = dict(title = 'Anzahl'))
# Plot erstellen
fig = go.Figure(data=[trace], layout=layout)
iplot(fig)

## Variablen löschen
del fig, layout, data, trace 

***
## 3.4. Wann sind die Filme bewertet worden?</a>
***

In [ ]:
# Daten einholen
data = df_rating['Date'].value_counts()
data.index = pd.to_datetime(data.index)
data.sort_index(inplace=True)

# Linie erstellen
trace = go.Scatter(x = data.index,
                   y = data.values,
                   marker = dict(color = '#db0000'))
# Layout erstellen
layout = dict(title = '{} Movie-Rating gruppiert mit dem Attribut "Date"'.format(df_rating.shape[0]),
              xaxis = dict(title = 'Datum'),
              yaxis = dict(title = 'Ratings'))

# Plot erstellen
fig = go.Figure(data=[trace], layout=layout)
iplot(fig)

## Delete Variables
del fig, layout, data, trace 

***
## 3.5. Verteilung von Filmbewertungen und Benutzern </a>
***

In [ ]:
##### Bewertungen je Film #####
# Daten einholen
data = df_rating.groupby('Movie_Id')['Rating'].count().clip(upper=9999)

# Linie erstellen
trace = go.Histogram(x = data.values,
                     name = 'Ratings',
                     xbins = dict(start = 0,
                                  end = 10000,
                                  size = 100),
                     marker = dict(color = '#db0000'))
# Layout erstellen
layout = go.Layout(title = 'Distribution Of Ratings Per Movie (Clipped at 9999)',
                   xaxis = dict(title = 'Ratings Per Movie'),
                   yaxis = dict(title = 'Count'),
                   bargap = 0.2)

# Plot erstellen
fig = go.Figure(data=[trace], layout=layout)
iplot(fig)

del fig, layout, data, trace


##### Ratings je User #####
# Daten einholen
data = df_rating.groupby('Cust_Id')['Rating'].count().clip(upper=299)

# Linie erstellen
trace = go.Histogram(x = data.values,
                     name = 'Ratings',
                     xbins = dict(start = 0,
                                  end = 1000,
                                  size = 2),
                     marker = dict(color = '#db0000'))
# Layout erstellen
layout = go.Layout(title = 'Distribution Of Ratings Per User (Clipped at 199)',
                   xaxis = dict(title = 'Ratings Per User'),
                   yaxis = dict(title = 'Count'),
                   bargap = 0.2)

# Plit erstelle
fig = go.Figure(data=[trace], layout=layout)
iplot(fig)

## Delete Variables
del fig, layout, data,  trace 

In [ ]:
### Alternative Darstellung


fig, ax = plt.subplots(1, 2, figsize=(14, 6))

data = df_rating.groupby('Cust_Id')['Rating'].count()
sns.distplot(data[data  < 200], kde=False, ax=ax[0]);
sns.distplot(data[data  > 200], kde=False, ax=ax[1], bins=[i for i in range(100,1000,20)]);

del fig, ax, data

In [ ]:
##Alternative Darstellung

fig, ax = plt.subplots(1, 2, figsize=(14, 6))

data = df_rating.groupby('Movie_Id')['Rating'].count()


sns.histplot(data[data  < 10000], kde=False, ax=ax[0], bins=[i for i in range(0,10000,200)]);
sns.histplot(data[data  > 10000], kde=False, ax=ax[1], bins=[i for i in range(10000,150000,5000)]);


del fig, ax, data

***
# <a id=4> 4. Filtern der Daten</a>
***

In [ ]:
# Filtern von Mindestanzahl von Ratings
# Derzeit werden keine Filme gefiltert.
min_movie_ratings = 0
filter_movies = (df_rating['Movie_Id'].value_counts()>min_movie_ratings)
filter_movies = filter_movies[filter_movies].index.tolist()

# Filtern von Usern die weniger als min_user_ratings bewertet haben
min_user_ratings = 400
filter_users = (df_rating['Cust_Id'].value_counts()>min_user_ratings)
filter_users = filter_users[filter_users].index.tolist()

# Actual filtering
df_filterd = df_rating[(df_rating['Movie_Id'].isin(filter_movies)) & (df_rating['Cust_Id'].isin(filter_users))]

print('Shape User-Ratings ungefiltert:\t{}'.format(df_rating.shape))
print('Shape User-Ratings gefiltert:\t{}'.format(df_filterd.shape))

# Entfernen der Spalte Datum aus 
df_filterd = df_filterd.drop('Date', axis= 1)

# Löschen der Variablen
del filter_movies, filter_users, min_movie_ratings, min_user_ratings

***
# <a id=5>5. Anwenden des KNN-Modells</a>
***


## 5.1 Vorbereitungen für das KNN-Modell</a>
***

In [ ]:
# Entfernen der Spalte Datum aus 
df_filterd = df_filterd.drop('Date', axis= 1)

# Erstellen eines Film-Recommendation Dataframe und den Distance
df_recommendation = pd.DataFrame(columns=['Movie_Id', 'R_Movie_Id_1', 'R_Distance_1' , 'R_Movie_Id_2'  , 'R_Distance_2' , 'R_Movie_Id_3'  ,'R_Distance_3','R_Movie_Id_4' ,'R_Distance_4' ,'R_Movie_Id_5' ,'R_Distance_5','R_Movie_Id_6' ,'R_Distance_6'])
Convert_dic= {'Movie_Id': 'int32', 'R_Movie_Id_1': 'int32','R_Distance_1':'float','R_Movie_Id_2': 'int32','R_Distance_2':'float','R_Movie_Id_3': 'int32','R_Distance_3':'float','R_Movie_Id_4': 'int32','R_Distance_4':'float','R_Movie_Id_5': 'int32','R_Distance_5':'float', 'R_Movie_Id_6': 'int32','R_Distance_6':'float'}
df_recommendation = df_recommendation.astype(Convert_dic)

# Löschen der Variablen
del Convert_dic

***
## 5.1.1 Bruce Model</a>
***

In [ ]:
# Erstellen der Pivotmatrix movie-user, alle nicht vorhandenen Werte werden mit Null aufgeügllt
movie_user_mat = df_filterd.pivot(index='Movie_Id', columns='Cust_Id', values='Rating').fillna(0)


# Übergeben der Parameter für KNN
user_movie_table_matrix = csr_matrix(movie_user_mat)

# Übergeben der Parameter für KNN
model_knn = NearestNeighbors(metric = 'cosine', algorithm = 'brute',n_jobs=-1, n_neighbors= 7)
model_knn.fit(user_movie_table_matrix)

### Erstellen der Arrays Distance (Metrik) und Distanz (Nachbar)
distances, indices = model_knn.kneighbors(user_movie_table_matrix)


In [ ]:
## Index um 1 erhöhen und Exportieren

Dateipfad = './Nearest_Neighbor.csv'
pd.DataFrame(indices).to_csv(Dateipfad, index=False, header=False)
df_nachbarn = pd.read_csv(Dateipfad, names = ['ID', 'n_1','n_2','n_3','n_4','n_5','n_6'])
#Erhöht jeden Wert in der Spalte um 1, muss angepasst werden, wenn die n_neighbors angepasst werden.
for index, row in df_nachbarn.iterrows():

    row['ID']+=1
    row['n_1']+=1
    row['n_2']+=1
    row['n_3']+=1
    row['n_4']+=1
    row['n_5']+=1
    row['n_6']+=1

pd.DataFrame(df_nachbarn).to_csv('./Nearest_Neighbor+1.csv', sep=',', index=False, header=True)

del  row, index, 

***
## 5.2 Export der nächsten Nacharn</a>
***
Dieser Abschnitt kann ignoriert werden und dient nur der Aufbereitung und Verarbeitung der Daten

In [ ]:
#Erstellen eines leeren Dataframes

df_recommendation = pd.DataFrame(columns=['Movie_Id', 'R_Movie_Id_1', 'R_Distance_1' , 'R_Movie_Id_2'  , 'R_Distance_2' , 'R_Movie_Id_3'  ,'R_Distance_3','R_Movie_Id_4' ,'R_Distance_4' ,'R_Movie_Id_5' ,'R_Distance_5','R_Movie_Id_6' ,'R_Distance_6'])
Convert_dic= {'Movie_Id': 'int32', 'R_Movie_Id_1': 'int32','R_Distance_1':'float','R_Movie_Id_2': 'int32','R_Distance_2':'float','R_Movie_Id_3': 'int32','R_Distance_3':'float','R_Movie_Id_4': 'int32','R_Distance_4':'float','R_Movie_Id_5': 'int32','R_Distance_5':'float', 'R_Movie_Id_6': 'int32','R_Distance_6':'float'}
df_recommendation = df_recommendation.astype(Convert_dic)

In [ ]:
# Erstellen einer Liste mit allen vorhandenen Movie_IDs in der Movie-User Matrix, diese dient als
movie_index_list = list(movie_user_mat.index.values)

### Durch das Iterieren der Daten werden 6 Empfehlungen in ein Dataframe geschrieben.
for  x in movie_index_list:

    movie = []
    distance = []
        

    for i in range(0, len(distances.flatten())):
        if i != 0:
            movie.append(movie_user_mat.index[indices.flatten()[i]])
            distance.append(distances.flatten()[i])    

    m=pd.Series(movie,name='Movie_Id')
    d=pd.Series(distance,name='distance')
    recommend = pd.concat([m,d], axis=1)
    recommend = recommend.sort_values('distance',ascending=False)

    # Anfügen der Empfehlungen für jeden Film in 
    df_recommendation = df_recommendation.append({'Movie_Id': movie_id, 'R_Movie_Id_1': recommend['Movie_Id'].iloc[0], 'R_Distance_1':recommend['distance'].iloc[0],
        'R_Movie_Id_2': recommend['Movie_Id'].iloc[1], 'R_Distance_2':recommend['distance'].iloc[1],
        'R_Movie_Id_3': recommend['Movie_Id'].iloc[2], 'R_Distance_3':recommend['distance'].iloc[2],
        'R_Movie_Id_4': recommend['Movie_Id'].iloc[3], 'R_Distance_4':recommend['distance'].iloc[3],
        'R_Movie_Id_5': recommend['Movie_Id'].iloc[4], 'R_Distance_5':recommend['distance'].iloc[4],
        'R_Movie_Id_6': recommend['Movie_Id'].iloc[5], 'R_Distance_6':recommend['distance'].iloc[5]
        },ignore_index=True)
        

In [ ]:
df_nachbarn = df_recommendation.drop(['R_Distance_1', 'R_Distance_2', 'R_Distance_3', 'R_Distance_4', 'R_Distance_5', 'R_Distance_6'],axis=1)
df_nachbarn.to_csv('./Nearest_Neighbor_brute.csv',index=False, sep=',',header= 'True')

In [ ]:
# Dataframe Types bearbeiten
Convert_dic= {'Movie_Id': 'int32', 'R_Movie_Id_1': 'int32','R_Distance_1':'float','R_Movie_Id_2': 'int32','R_Distance_2':'float','R_Movie_Id_3': 'int32','R_Distance_3':'float','R_Movie_Id_4': 'int32','R_Distance_4':'float','R_Movie_Id_5': 'int32','R_Distance_5':'float', 'R_Movie_Id_6': 'int32','R_Distance_6':'float'}
df_recommendation = df_recommendation.astype(Convert_dic)

# Exportieren der Nächsten Nachbarn
df_nachbarn =df_recommendation.drop(['R_Distance_1', 'R_Distance_2', 'R_Distance_3', 'R_Distance_4', 'R_Distance_5', 'R_Distance_6'], axis= 1)
df_nachbarn.to_csv('./Nearest_Neighbor_brute.csv',index= False, sep=',',header='True')

# Exportieren der Nächsten Nachbarn davon die Distanz
df_distance =df_recommendation.drop(['R_Movie_Id_1', 'R_Movie_Id_2', 'R_Movie_Id_3', 'R_Movie_Id_4', 'R_Movie_Id_5', 'R_Movie_Id_6'], axis= 1)
df_distance.to_csv('./Nearest_Neighbor_distance_brute.csv',index= False, sep=',',header='True')

# Export der Empfehlungen und IDs mit Rundungen als CSV
df_rounded = df_recommendation.round({'R_Distance_1': 4, 'R_Distance_2':4,'R_Distance_3': 4, 'R_Distance_4':4,'R_Distance_5':4,'R_Distance_6':4 })
df_rounded.to_csv('./Movie_Recommendations_header_IDs_rounded_brute.csv',index= False, sep=';',header='True', decimal=',')

# Mappen der Movie_IDs and die Datei Movie_Titles

df_rounded['Movie_Id'] = df_rounded['Movie_Id'].map(df_movie_titles.set_index('Movie_Id')['Name'])
df_rounded['R_Movie_Id_1'] = df_rounded['R_Movie_Id_1'].map(df_movie_titles.set_index('Movie_Id')['Name'])
df_rounded['R_Movie_Id_2'] = df_rounded['R_Movie_Id_2'].map(df_movie_titles.set_index('Movie_Id')['Name'])
df_rounded['R_Movie_Id_3'] = df_rounded['R_Movie_Id_3'].map(df_movie_titles.set_index('Movie_Id')['Name'])
df_rounded['R_Movie_Id_4'] = df_rounded['R_Movie_Id_4'].map(df_movie_titles.set_index('Movie_Id')['Name'])
df_rounded['R_Movie_Id_5'] = df_rounded['R_Movie_Id_5'].map(df_movie_titles.set_index('Movie_Id')['Name'])
df_rounded['R_Movie_Id_6'] = df_rounded['R_Movie_Id_6'].map(df_movie_titles.set_index('Movie_Id')['Name'])

# Exportieren der Empfehlungen mit Filmnamen
df_rounded.to_csv('/Movie_Recommendations_header_Titels_rounded.csv',index= False, sep=';',header='True', decimal=',')

#löschen der Variable

del df_rounded, df_nachbarn

***
##  5.3 Vorbereitung für Precision and Recall </a>
***

In [ ]:
## Um die Laufzeit zu verbessern, werden mit dieser Funktion die zuvor ausgegebenen Distances und Nearest Neighbor mit Hilfe einer CSV eingelesen

def empfehlung_movie (best_movie):
    # Leere arrays erstellen
    nachbarn = []
    index = best_movie

    df_nachbarn = pd.read_csv('./Nearest_Neighbor+1.csv')
    l_nachbarn = df_nachbarn.iloc[index-1]
    l_nachbarn = list(l_nachbarn.tolist())[:0:-1]
    return l_nachbarn


***
##  5.4 Precision and Recall </a>
***

In [ ]:


[1]# Import Modules
import time
# Runtime variable
start = time.time()

### Ausklammern wenn oben bereits iteriert wurde
import json
import numpy as np
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from statistics import mean
#### 

#Final_Code_Abgabe import empfehlung_movie --> Wenn das als Py Datei ausgeführt werden soll

def get_user_ids_to_drop():
    user_ids_to_drop = []
    f = open('./testset.json')
    data = json.load(f)
    for i in data:
        user_ids_to_drop.append(i['User_Id'])
    return user_ids_to_drop

def get_calculation_base(raw_true, raw_pred):
    boolean_true = []
    boolean_pred = []
    for i in range(17770):
        boolean_true.append(False)
        boolean_pred.append(False)
    for i in raw_true:
        boolean_true[i-1] = True
    for i in raw_pred:
        boolean_pred[int(i-1)] = True
    return boolean_true, boolean_pred

def get_mean_precision_recall(): 
    # Opening JSON file
    f = open('./testset.json')
    
    # returns JSON object as a dictionary
    data = json.load(f)

    # Iterating through the dictionary
    precision_total = []
    recall_total = []
    for i in data:
        #################################################################
        # Get predictions for the prediction base
        raw_pred = []
        for j in i['Prediction_Base']:
            raw_pred = np.append(raw_pred, empfehlung_movie(j))
        #################################################################
        # Get true values for the prediction base
        raw_true = i['Raw_true']

        # Get precision and recall for particular testdata
        boolean_true, boolean_pred = get_calculation_base(raw_true, raw_pred)
        precision = precision_score(y_true = boolean_true, y_pred = boolean_pred)
        recall = recall_score(y_true = boolean_true, y_pred = boolean_pred)
        precision_total = np.append(precision_total,precision)
        recall_total = np.append(recall_total,recall)

    return mean(precision_total), mean(recall_total)

mean_precision, mean_recall = get_mean_precision_recall()
print(mean_precision)
print(mean_recall)

[8]# Runtime analysis
end = time.time()
print('Runtime: {:5.3f}s'.format(end-start))

#Löschen von Variablen
del end, start, mean_precision, mean_recall

***
##  5.5. Empfehlung mit Hilfe des Fuzzy Algorithmus </a>
***

In [ ]:
# Erstellen einer Mappers von IDs zu Filmtitel
movie_to_idx = {
    movie: i for i, movie in 
    enumerate(list(df_movie_titles.set_index('Movie_Id').loc[movie_user_mat.index].Name))}

In [ ]:
def fuzzy_matching(mapper, fav_movie, verbose=True):
    """
    gibt die nächstliegende Übereinstimmung per Fuzzy zurück. Wenn keine Übereinstimmung gefunden wird, wird None zurückgegeben.

    Rückgabe
    ------
    Index der nächstgelegenen Übereinstimmung
    """
    match_tuple = []
    # Suchen vom Match
    for title, idx in mapper.items():
        ratio = fuzz.ratio(title.lower(), fav_movie.lower())
        if ratio >= 60:
            match_tuple.append((title, idx, ratio))
    # Sortieren der Matches
    match_tuple = sorted(match_tuple, key=lambda x: x[2])[::-1]
    if not match_tuple:
        print('Kein Film gefunden')
        return
    if verbose:
        print('Folgende Filme wurden in der Datenbank gefunden: {0}\n'.format([x[0] for x in match_tuple]))
    return match_tuple[0][1]



def make_recommendation(model_knn, data, mapper, fav_movie, n_recommendations):
    # Fitting
    model_knn.fit(data)
    # Ausgabe des Index
    print('Deine Eingabe war:', fav_movie)
    idx = fuzzy_matching(mapper, fav_movie, verbose=True)
    distances, indices = model_knn.kneighbors(data[idx], n_neighbors=n_recommendations+1)
    # Holen der Index -- Film Liste
    raw_recommends = \
        sorted(list(zip(indices.squeeze().tolist(), distances.squeeze().tolist())), key=lambda x: x[1])[:0:-1]
    # Reversives Mapping
    reverse_mapper = {v: k for k, v in mapper.items()}
    # Ausgabe
    print('Empfehlung für {}:'.format(fav_movie))
    for i, (idx, dist) in enumerate(raw_recommends):
        print('{0}: {1}, mit Distanz auf {2}'.format(i+1, reverse_mapper[idx], dist))

In [ ]:
my_favorite = '  <<Type Your Movie Name >>'

make_recommendation(
    model_knn=model_knn,
    data=user_movie_table_matrix,
    fav_movie=my_favorite,
    mapper=movie_to_idx,
    n_recommendations=5)